# One time replication tables between database using pandas 

### copy from source to target / no processing - naive approce 
### implemeted over sqlite but works with any db 

for questions and comments me.geva@gmail.com 

In [37]:
import sqlite3
import pandas as pd

In [45]:
conn_orig = sqlite3.connect('/Users/geva/db/chinook.db')
conn_dest = sqlite3.connect('/Users/geva/db/chinook_target.db')

In [46]:
# get list of tables for mysql use "show tables"
qry_list_tables = """SELECT name FROM sqlite_schema WHERE type ='table' AND name NOT LIKE 'sqlite_%'"""
df_list_tables = pd.read_sql(con=conn_orig,sql=qry_list_tables)
#the tables to be copeid 
df_list_tables

,name
0,albums
1,artists
2,customers
3,employees
4,genres
5,invoices
6,invoice_items
7,media_types
8,playlists
9,playlist_track


In [47]:
exclude_tables = ['tmp_data']

In [48]:
#looping over the tables and create in the far end
for n,tab_name in df_list_tables['name'].iteritems():
    if tab_name not in exclude_tables:
        print(f"moving table: {tab_name}")
        qry_get_data = f"select * from {tab_name}"
        df_get_data = pd.read_sql(con=conn_orig,sql=qry_get_data)
        try:
            df_get_data.to_sql(con=conn_dest,name=tab_name)
        except ValueError:
            print (f"table {tab_name} already exists")
    else:
        print(f'skipping table {tab_name} - in exclude_tables')

moving table: albums
moving table: artists
moving table: customers
moving table: employees
moving table: genres
moving table: invoices
moving table: invoice_items
moving table: media_types
moving table: playlists
moving table: playlist_track
moving table: tracks
skipping table tmp_data - in exclude_tables


In [50]:
# verification - Count target row  
df_tables_count=pd.DataFrame({'table_name':[],'cnt':[]})
for n,tab_name in df_list_tables['name'].iteritems():
    if tab_name not in exclude_tables:
        df_tables_count=df_tables_count.append(pd.read_sql(con=conn_dest,sql=f"select '{tab_name}' table_name, count(*) cnt from {tab_name}"))
df_tables_count['cnt'] = df_tables_count['cnt'].astype(int)
df_tables_count

,table_name,cnt
0,albums,347
0,artists,275
0,customers,59
0,employees,8
0,genres,25
0,invoices,412
0,invoice_items,2240
0,media_types,5
0,playlists,18
0,playlist_track,8715


In [51]:
conn_dest.close()
conn_orig.close()